In [1]:
import tensorflow as tf

In [2]:
from transformers import AutoModel, AutoTokenizer
model_name = "csebuetnlp/banglabert"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
import torch
import torch.nn as nn
from transformers import AutoModel

class BanglaBERT_LSTM(nn.Module):
    def __init__(self, lstm_hidden_size=256, lstm_layers=1, dropout=0.3):
        super(BanglaBERT_LSTM, self).__init__()

        self.bert = model
        self.lstm = nn.LSTM(input_size=768, hidden_size=lstm_hidden_size, num_layers=lstm_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(lstm_hidden_size * 2, 1)  
        self.dropout = nn.Dropout(dropout)
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        # Get embeddings from BanglaBERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # (batch_size, seq_len, 768)

        # Pass through BiLSTM
        lstm_out, _ = self.lstm(hidden_states)  # (batch_size, seq_len, hidden_size*2)
        lstm_out = lstm_out[:, -1, :]  # Take the last time step output

        # Pass through the classifier
        logits = self.fc(self.dropout(lstm_out))  # (batch_size, 1)

        # Compute loss if labels are provided
        loss = None
        if labels is not None:
            labels = labels.float().unsqueeze(1)  
            loss = self.criterion(logits, labels)

        return (loss, logits) if loss is not None else logits

In [4]:
model = BanglaBERT_LSTM()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


In [5]:
model.load_state_dict(torch.load("/kaggle/input/unber-1k/BB-LSTM.pth"))

<ipython-input-5-6e2c09698994>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/input/unber-1k/BB-LSTM.pth"))


<All keys matched successfully>

In [53]:
model1 = AutoModel.from_pretrained(model_name, num_labels=3)

In [54]:
class BanglaBERT_LSTM_Multiclass(nn.Module):
    def __init__(self, lstm_hidden_size=256, lstm_layers=1, dropout=0.3, num_classes=3):
        super(BanglaBERT_LSTM_Multiclass, self).__init__()

        self.bert = model1  # Example, change to correct model

        self.lstm = nn.LSTM(input_size=768, hidden_size=lstm_hidden_size, num_layers=lstm_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)

        self.fc = nn.Linear(lstm_hidden_size * 2, num_classes)  

        self.dropout = nn.Dropout(dropout)

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        # Get embeddings from BanglaBERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # (batch_size, seq_len, 768)

        # Pass through BiLSTM
        lstm_out, _ = self.lstm(hidden_states)  # (batch_size, seq_len, hidden_size*2)
        lstm_out = lstm_out[:, -1, :]  # Take the last time step output

        # Pass through the classifier
        logits = self.fc(self.dropout(lstm_out))  # (batch_size, num_classes)
        # Compute loss if labels are provided
        loss = None
        if labels is not None:
            loss = self.criterion(logits, labels)
            
        return (loss, logits) if loss is not None else logits

In [55]:
model1 = BanglaBERT_LSTM_Multiclass()

In [56]:
model1.load_state_dict(torch.load("/kaggle/input/unber-1k/BB-LSTM-1.pth"))

<ipython-input-56-1e0c26abe40f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load("/kaggle/input/unber-1k/BB-LSTM-1.pth"))


<All keys matched successfully>

In [57]:
model.eval()

BanglaBERT_LSTM(
  (bert): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps

In [58]:
model1.eval()

BanglaBERT_LSTM_Multiclass(
  (bert): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(

In [12]:
text = [
  "দইরে টাইনে ছিরে ফেলাই দিবো",
  "কেও হোটেলের মাল কে খাওয়ায় দিলো না",
  "বোকা** ঘুমাচ্ছে",
  "রোজার মাসে বের করে দিছে",
  "বেয়াদব বেডা আমার গায়ে হাত দিছে",
  "তুমি কথা দিয়েছিলে কখনো ছেড়ে যাবে না! কোথায় গেলো তোমার সেই কথা আর কোথাই বা তুমি...!!",
  "ব্যর্থ প্রেম - Feeling sad তুমি তো ভালো না তোমার জন্য এত কষ্ট হয়",
  "Koi tui? Tk dibi na Onek din to hoya geche Oda ma**dari",
  "আমার Jelous...! আমার Loyalty",
  "এক সপ্তাহ ছুটির পর অফিস এসে ওয়ার্ক লোড দেখার পর আমি",
  "দোস্ত চল পাহাড়ে ট্রেকিং- এ যাই আইলসা ফ্রেন্ড: এমন সুবিধা আছে?",
  "POV: আমরা দুই বোন আমাদের কোনো ভাই নাই"
]

In [82]:
from datasets import Dataset
import pandas as pd

def predict_uniclass(text):
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
  inputs = {key: value.to(device) for key, value in inputs.items()} 

  with torch.no_grad():
      logits = model(inputs['input_ids'], inputs['attention_mask'])

  logits = logits.squeeze(0)
  probabilities = torch.sigmoid(logits).tolist()

  labels = ['Safe', 'Unsafe']
  pred_val = [1 if probabilities[0] >= 0.5 else 0]

  return labels[pred_val[0]]

In [83]:
predict_uniclass(text[0])

'Unsafe'

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)  # Move model to the correct device

def predict_multiclass(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()} 

    with torch.no_grad():
        logits = model1(inputs['input_ids'], inputs['attention_mask'])

    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    labels = ['Adult', 'Harmful', 'Suicidal']

    return labels[predicted_class]

In [91]:
def predict(text):
    pred = ""
    mpred = ""
    pred = predict_uniclass(text)
    if pred=='Unsafe':
        mpred = predict_multiclass(text)
    
    print(f"Text: {text}")
    if pred=="Unsafe":
        print(f"Category: {pred}, Class: {mpred}")
    else:
        print(f"Category: {pred}")

In [93]:
for txt in text:
    print(predict(txt))

Text: দইরে টাইনে ছিরে ফেলাই দিবো
Category: Unsafe, Class: Harmful
None
Text: কেও হোটেলের মাল কে খাওয়ায় দিলো না
Category: Unsafe, Class: Adult
None
Text: বোকা** ঘুমাচ্ছে
Category: Unsafe, Class: Adult
None
Text: রোজার মাসে বের করে দিছে
Category: Unsafe, Class: Adult
None
Text: বেয়াদব বেডা আমার গায়ে হাত দিছে
Category: Unsafe, Class: Adult
None
Text: তুমি কথা দিয়েছিলে কখনো ছেড়ে যাবে না! কোথায় গেলো তোমার সেই কথা আর কোথাই বা তুমি...!!
Category: Unsafe, Class: Suicidal
None
Text: ব্যর্থ প্রেম - Feeling sad তুমি তো ভালো না তোমার জন্য এত কষ্ট হয়
Category: Unsafe, Class: Suicidal
None
Text: Koi tui? Tk dibi na Onek din to hoya geche Oda ma**dari
Category: Unsafe, Class: Adult
None
Text: আমার Jelous...! আমার Loyalty
Category: Safe
None
Text: এক সপ্তাহ ছুটির পর অফিস এসে ওয়ার্ক লোড দেখার পর আমি
Category: Safe
None
Text: দোস্ত চল পাহাড়ে ট্রেকিং- এ যাই আইলসা ফ্রেন্ড: এমন সুবিধা আছে?
Category: Unsafe, Class: Harmful
None
Text: POV: আমরা দুই বোন আমাদের কোনো ভাই নাই
Category: Unsafe, Class: Adult
None
